In [31]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [53]:
import torch
import torchvision

# dataset preparation
import torchvision.transforms as transforms

# NN
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models as torch_models

from torchsummary import summary

# Utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Other
import os
import zipfile
import requests
import hashlib
import random
import time
import datetime
import json
import PIL
from PIL import Image

np.random.seed(5051)

device = "cuda" if torch.cuda.is_available() else "cpu"


num_embeddings = 10000 # look num_words in file "prepare_ds.py"
                             # or it must be len(SrachSet.mydict.keys()) == 229520 ???

# Подготовка датасета

Надо что-нибудь написать... Потом... Как-нибудь...

In [33]:
# Делаем свой кастомный датасет


class SrachSet(torch.utils.data.Dataset):
    mydict = None
    
    def __init__(self, _type: str, _path: str = "out.csv", _json_path: str = "word_index_dict.json"):
        self.__type = _type
        assert self.__type in ["train", "valid", "test"]
        self.mypath = _path
        self.mypath_dict = _json_path
        self.laeblo = []
        self.myds = []
        
        df = pd.read_csv(self.mypath)
        df = df.reset_index()
        for i, row_i in df.iterrows():
            self.laeblo.append(   [row_i["toxics"], row_i["obscene"], row_i["threat"], row_i["insult"], row_i["identity_hate"]]   )
            self.myds.append  (   list(map(int, str(row_i["comment"]).split()))   )
        
        if(self.__type == "train"):
            self.myds = self.__split_ds(self.myds)[0]
            self.laeblo = self.__split_ds(self.laeblo)[0]
        elif(self.__type == "valid"):
            self.myds = self.__split_ds(self.myds)[1]
            self.laeblo = self.__split_ds(self.laeblo)[1]
        elif(self.__type == "test"):
            self.myds = self.__split_ds(self.myds)[2]
            self.laeblo = self.__split_ds(self.laeblo)[2]
        else:
            raise Exception("wtf")
        random.shuffle(self.myds)

        if(SrachSet.mydict == None):
            S = None
            with open(self.mypath_dict, 'r', encoding="utf-8") as temp:
                S = temp.read()
            buff = json.loads(S)
            SrachSet.mydict = {v: k for k, v in buff.items()}
        
    def __split_ds(self, a: list) -> "(list, list, list)":
        split_num = int(0.7*len(a) +0.5)
        train = a[:split_num]               # 70 %
        test_and_valid  = a[split_num:]     # 30 %
        
        split_num = int((1/3)*len(test_and_valid) +0.5)
        valid = test_and_valid[:split_num]  # 10% = 1/3 * 30%
        test = test_and_valid[split_num:]   # 20% = 2/3 * 30%
        return (train, valid, test)

    def __len__(self):
        return len(self.myds)

    def __getitem__(self, idx):
        cur = self.myds[idx]
        wa  = self.laeblo[idx]

        #tenz_cur = torch.LongTensor(cur)
        #cur = torch.Tensor(cur)
        cur = torch.IntTensor(cur)
        wa = torch.IntTensor(wa)
        
        return cur, wa

    def convert_tensor_to_label(tensor: "Tensor"):
        res = ["toxics: ", "obscene: ", "threat: ", "insult: ", "identity_hate: "]
        li = 0
        for el_i in tensor:
            if el_i != 0:
                res[li] = res[li] + "yes"
            else:
                res[li] = res[li] + "no"
            li-=-1
        
        return res

    def show_seq(seq_tensor: "Tensor") -> str:
        S = ""
        for item_i in seq_tensor:
            kye = int(item_i)
            if(kye != 0):
                S += f"{SrachSet.mydict[kye]} "
        return S


In [34]:
# функция инициализации датасетлоудеров:
trainloader, validloader, testloader = None, None, None
def dsloaders_init(batch_size):
    global trainloader
    global validloader
    global testloader
    mytrainset = SrachSet("train")
    trainloader = torch.utils.data.DataLoader(mytrainset, batch_size=batch_size, num_workers=2)

    myvalidset = SrachSet("valid")
    validloader = torch.utils.data.DataLoader(myvalidset, batch_size=batch_size, num_workers=2)

    mytestset = SrachSet("test")
    testloader = torch.utils.data.DataLoader(mytestset, batch_size=batch_size, num_workers=2)

In [35]:
# функция показа содержимого
def show_srach(srach_rensor: "Tensor", label_tensor: "Tensor") -> None:
    srach_list = [int(i) for i in srach_rensor]
    print(srach_list)
    print("<--->")
    print(SrachSet.show_seq(srach_rensor))
    print(f"It is: {SrachSet.convert_tensor_to_label(label_tensor)}")
    print("\n\n")

In [36]:
dsloaders_init(16)

In [38]:
# получаем несколько случайных обучающих $%#5^
dataiter = iter(trainloader)
seqs_tenz, labels_tenz = next(dataiter)

for seq_tenz_i, label_tenz_i in zip(seqs_tenz, labels_tenz):
    show_srach(seq_tenz_i, label_tenz_i)

[170, 380, 1, 258, 8, 5, 1764, 17, 1, 743, 218, 312, 26, 6, 60, 267, 77, 9, 10, 517, 2669, 12, 290, 117, 93]
<--->
hi clearly the subject is a model as the external links show but i don't understand why that in itself consideration for speedy deletion thanks 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2562]
<--->
and liar 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[146, 2, 120, 619, 50, 114, 15, 90, 46, 122, 36, 250, 17, 50, 18, 64, 2, 2028, 27, 372, 4, 14, 12, 25, 4489]
<--->
right to say whatever they want on their talk pages so long as they are here to build an encyclopedia and not for or motives 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[369, 9, 3725, 8, 1, 1043, 3, 843, 49, 51, 40, 18, 843, 62, 34, 14, 982, 55, 3, 1, 1385, 441, 10, 1, 956]
<--->
saying that judaism is th

In [39]:
# получаем несколько случайных валидационных ?!№#@
dataiter = iter(validloader)
seqs_tenz, labels_tenz = next(dataiter)

for seq_tenz_i, label_tenz_i in zip(seqs_tenz, labels_tenz):
    show_srach(seq_tenz_i, label_tenz_i)

[400, 111, 17, 2778, 4, 1209, 1211, 234, 9, 71, 14, 1074, 186, 38, 172, 1789, 33, 26, 6, 43, 454, 7, 5792, 11, 209]
<--->
users such as sockpuppets and spam accounts i'll that i'm not entirely sure what you're playing at but i would suggest you knock it off 
It is: ['toxics: yes', 'obscene: yes', 'threat: no', 'insult: yes', 'identity_hate: no']



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 65, 6, 86, 19, 4628, 59, 3, 1, 665, 20, 1, 6346, 3996]
<--->
i think i may have solved some of the problems with the ethical slut 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[4, 371, 76, 10, 1710, 2529, 2320, 20, 1, 760, 10, 255, 69, 371, 10, 255, 26, 129, 13, 133, 8, 286, 191, 33, 4678]
<--->
and written out in roman letters whereas with the series in english it's written in english but again this discussion is probably better at wt 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[19, 2161, 726, 15, 6773, 5132, 1924, 

In [40]:
# получаем несколько случайных тестовых %1№2!#?
dataiter = iter(testloader)
seqs_tenz, labels_tenz = next(dataiter)

for seq_tenz_i, label_tenz_i in zip(seqs_tenz, labels_tenz):
    show_srach(seq_tenz_i, label_tenz_i)

[819, 2, 617, 6, 65, 50, 18, 103, 97, 1020, 3, 2484, 192, 6, 79, 5993, 919, 17, 6, 3147, 155, 3, 1, 231, 4]
<--->
allowed to stay i think they are very good examples of tom however i am admittedly bias as i invented 2 of the 3 and 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[103, 259, 2534, 5138, 10, 393, 3377, 5, 951, 386, 48, 63, 29, 635, 2534, 28, 171, 1221, 934, 12, 422, 137, 7, 12, 839]
<--->
very few aircraft appearances in media warrant a special mention please see wikipedia wikiproject aircraft page content popular culture for guidelines thank you for contributing 
It is: ['toxics: no', 'obscene: no', 'threat: no', 'insult: no', 'identity_hate: no']



[630, 8, 52, 8, 2911, 531, 4859, 4, 828, 5331, 2029, 169, 461, 29, 397, 898, 1235, 1307, 322, 6, 16, 3060, 854, 1474, 2420]
<--->
guy is he is deliberately removing punctuation and creating grammatical mistakes http en wikipedia org w index php title i be alive action diff oldid 
It i

In [41]:
# Смотрим, что вообще есть видеокарта

print(f"CUDA? {torch.cuda.is_available()}")
print(f"Devices count: {torch.cuda.device_count()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"Device: {torch.cuda.device(0)}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


CUDA? True
Devices count: 1
Current device: 0
Device: <torch.cuda.device object at 0x7faf7293bf40>
GPU name: NVIDIA GeForce GTX 1070 Ti
Allocated: 0.0 GB
Cached:    0.0 GB


# Делаем собственную архитектуру

## Обучающий класс

In [46]:
# Модифицируем (видимо, не вперый раз) вспомогательный класс под текущую задачу
class Optimization:
    """Optimization is a helper class that allows training, validation, prediction.

    Optimization is a helper class that takes model, loss function, optimizer function
    learning scheduler (optional), early stopping (optional) as inputs. In return, it
    provides a framework to train and validate the models, and to predict future values
    based on the models.

    Attributes:
        model (nn.Module): Model class
        loss_fn (torch.nn.modules.Loss): Loss function to calculate the losses
        optimizer (torch.optim.Optimizer): Optimizer function to optimize the loss function
        train_losses (list[float]): The loss values from the training
        val_losses (list[float]): The loss values from the validation
    """
    def __init__(self, model, loss_fn, optimizer):
        """
        Args:
            model (nn.Module): Model class
            loss_fn (torch.nn.modules.Loss): Loss function to calculate the losses
            optimizer (torch.optim.Optimizer): Optimizer function to optimize the loss function
        """
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        self.err_rates = []
        
    def train_step(self, x, y) -> "loss":
        """The method train_step completes one step of training.

        Given the features (x) and the target values (y) tensors, the method completes
        one step of the training. First, it activates the train mode to enable back prop.
        After generating predicted values (yhat) by doing forward propagation, it calculates
        the losses by using the loss function. Then, it computes the gradients by doing
        back propagation and updates the weights by calling step() function.

        Args:
            x (torch.Tensor): Tensor for features to train one step
            y (torch.Tensor): Tensor for target values to calculate losses

        """
        # Sets model to train mode
        self.model.train() # dropout is here, so call it

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(yhat, y)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

    def train(self, train_loader, val_loader, n_epochs, test_loader = None) -> None:
        """The method train performs the model training

        The method takes DataLoaders for training and validation datasets, batch size for
        mini-batch training, number of epochs to train.
        Then, it carries out the training by iteratively calling the method train_step for
        n_epochs times. If early stopping is enabled, then it  checks the stopping condition
        to decide whether the training needs to halt before n_epochs steps. Finally, it saves
        the model in a designated file path.

        Args:
            train_loader (torch.utils.data.DataLoader): DataLoader that stores training data
            val_loader (torch.utils.data.DataLoader): DataLoader that stores validation data
            n_epochs (int): Number of epochs, i.e., train steps, to train
            test_loader (torch.utils.data.DataLoader): set it if need error_rate after one epoch
        """
        model_path = f'model_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}'

        start_time_gl = time.time()
        for epoch in range(1, n_epochs + 1):
            start_time_ep = time.time()
            
            gi, N = 0, len(train_loader)
            batch_losses = []
            print(f"({epoch}/{n_epochs}) {N}: ", end="")
            start_time_bt = time.time()
            for x_batch, y_batch in train_loader:

                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
                gi+=1
                print(f"_{gi}", end="")
            print(f" ({round(time.time() - start_time_bt, 2)} s)")
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.to(device)
                    y_val = y_val.to(device)
                    self.model.eval()       # dropout is here, so call it
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(yhat, y_val).item()
                    batch_val_losses.append(val_loss)
                    
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)
            
            # error_rate
            if(test_loader != None):
                preds, reals = self.evaluate(test_loader)

                correct_pred, total_pred = 0, 0
                for pred_i, real_i in zip(preds, reals):
                    for pred_i_j, real_i_j in zip(pred_i, real_i):
                        pred_i_j_one = np.argmax(pred_i_j)
                        if(pred_i_j_one == real_i_j):
                            correct_pred += 1
                        total_pred += 1
                error_rate = round(100 - correct_pred/total_pred*100, 2)
                self.err_rates.append(error_rate/100)
                
            print(f"({epoch}/{n_epochs}) Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}\t error rate: {error_rate if test_loader != None else None} %", end ="")
            print(f" ({round(time.time() - start_time_ep, 2)} s)")
        
        print(f"Train time: ({round(time.time() - start_time_gl, 2)} s)")
        torch.save(self.model.state_dict(), model_path)

    def evaluate(self, test_loader) -> "(predicted, values)":
        """The method evaluate performs the model evaluation

        The method takes DataLoaders for the test dataset, batch size for mini-batch testing,
        and number of features as inputs. Similar to the model validation, it iteratively
        predicts the target values and calculates losses. Then, it returns two lists that
        hold the predictions and the actual values.

        Note:
            This method assumes that the prediction from the previous step is available at
            the time of the prediction, and only does one-step prediction into the future.

        Args:
            test_loader (torch.utils.data.DataLoader): DataLoader that stores test data

        Returns:
            list[float]: The values predicted by the model
            list[float]: The actual values in the test set.

        """
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.to(device)
                y_test = y_test.to(device)
                self.model.eval()         # dropout is here, so call it
                yhat = self.model(x_test)
                predictions.append(yhat.to("cpu").detach().numpy())
                values.append(y_test.to("cpu").detach().numpy())

        return predictions, values

    def check_adequacy(self, test_loader) -> None:
        start_time = time.time()
        
        correct_pred = {classname: 0 for classname in doggyDS.get_classes_list()}
        total_pred = {classname: 0 for classname in doggyDS.get_classes_list()}

        preds, reals = self.evaluate(test_loader)

        for pred_i, real_i in zip(preds, reals):
            for pred_i_j, real_i_j in zip(pred_i, real_i):
                pred_i_j_one = np.argmax(pred_i_j)
                if(pred_i_j_one == real_i_j):
                    correct_pred[doggyDS.convert_tensor_to_labels([real_i_j])[0]] += 1
                total_pred[doggyDS.convert_tensor_to_labels([real_i_j])[0]] += 1

        accuracy_one_u = 0
        accuracy_one_d = 0
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            accuracy_one_u += correct_count
            accuracy_one_d += total_pred[classname]
            print(f'Accuracy for class: {classname:7s} is {accuracy:.1f} %')
        print(f'Accuracy for total: {100.0*accuracy_one_u/accuracy_one_d:.1f} %')
        print(f"Test time: ({round(time.time() - start_time, 2)} s)")
    
    def find_loss(model, valid_loader, lr_min, lr_max, steps=50) -> None:
        weight_decay = 0
        #dataiter = iter(testloader)
        #xs, ys = next(dataiter)
        #xs, ys = xs.to(device), ys.to(device)
        
        i, N = 0, steps
        X, Y = [], []
        print(f"{N}: ", end="")
        start_time = time.time()
        for i in range(N+1):
            cur_lr = lr_min + (lr_max-lr_min)*(i/N)
            loss_fn = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=cur_lr, weight_decay=weight_decay)
            opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
            #loss = opt.train_step(xs, ys)
            loss = 0
            for x_batch, y_batch in valid_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                loss_l = opt.train_step(x_batch, y_batch)
                loss += loss_l
            print(f"_{i+1}", end="")
            
            X.append(cur_lr)
            Y.append(loss)
        print(f" ({round(time.time() - start_time, 2)} s)")
        plt.plot(X, Y)
        #print(f"X={X}, Y={Y}")
        df = pd.DataFrame(data={"lr": X, "loss": Y})
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(df)
    
    def plot_losses(self):
        """The method plots the calculated loss values for training and validation
        """
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        if(len(self.err_rates) > 0):
            plt.plot(self.err_rates, label="Error rate")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

## Архитектура 1

In [72]:
class SrachNet1(nn.Module):
    def __init__(self):
        super().__init__()
        self.em1  = nn.Embedding(num_embeddings, 16).to(device)  
        #self.lstmcell2  = nn.LSTMCell(16, 32).to(device)
        self.lstm2  = nn.LSTM(16, 32).to(device)
        
        self.do3 = torch.nn.Dropout(0.3)
        
        self.dance4 = nn.Linear(32, 5).to(device)

    def forward(self, sentence):
        embeds = self.em1(sentence)
        #x = self.lstmcell2(x)
        lstm_out, _ = self.lstm2(embeds.view(len(sentence), 1, -1))
        #x = F.relu(x)
        #lstm_out = self.do3(lstm_out)
        x = self.dance4(lstm_out.view(len(sentence), -1))
        return x

    def compute_l1_loss(self, w): # Так или Адомом. Правда у него только L2...
        return torch.abs(w).sum()
    def compute_l2_loss(self, w): # Так или Адомом
        return torch.square(w).sum()
#net = Net()

In [60]:
# summary(SrachNet1(), (W, T, F))
torch.save(SrachNet1().state_dict(), "kokoko")

In [66]:
dsloaders_init(16)

In [75]:
model = SrachNet1()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
train_loader = trainloader

for x_batch, y_batch in train_loader:
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    
    model.zero_grad()
    model.train()
    yhat = model(x_batch)
    loss = loss_fn(yhat, y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
#     for x, y in zip(x_batch, y_batch):
#         model.train()

#         yhat = model(x)

#         loss = loss_fn(yhat, y)

#         loss.backward()

#         optimizer.step()
#         optimizer.zero_grad()


    print(loss.item())

RuntimeError: input.size(-1) must be equal to input_size. Expected 16, got 400

In [51]:
n_epochs = 3
learning_rate = 0.1
weight_decay = 0.00001 # for L2

model = SrachNet1()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(trainloader, validloader, n_epochs)
opt.plot_losses()

opt.check_adequacy(testloader)

(1/3) 5879: 

AssertionError: LSTMCell: Expected input to be 1-D or 2-D but received 3-D tensor